In [1]:
%reset 


Once deleted, variables cannot be recovered. Proceed (y/[n])? y


In [2]:
pwd

'/Users/bahar/Downloads/SL-Cloud-main2/DEPMAP_save_data'

In [29]:
import pandas as pd
import pandas_gbq as gbq
from google.cloud import bigquery
import numpy as np
import sys
sys.path.append('../scripts/')
import importlib
import BIGQUERY_operations
importlib.reload(BIGQUERY_operations)
from BIGQUERY_operations import *
import DEPMAP_data_preprocessing
importlib.reload(DEPMAP_data_preprocessing)
from DEPMAP_data_preprocessing import *

In [17]:
!gcloud auth application-default login

Traceback (most recent call last):
  File "/Users/bahar/Downloads/google-cloud-sdk/lib/gcloud.py", line 104, in <module>
    main()
  File "/Users/bahar/Downloads/google-cloud-sdk/lib/gcloud.py", line 100, in main
    sys.exit(gcloud_main.main())
  File "/Users/bahar/Downloads/google-cloud-sdk/lib/googlecloudsdk/gcloud_main.py", line 171, in main
    gcloud_cli = CreateCLI([])
  File "/Users/bahar/Downloads/google-cloud-sdk/lib/googlecloudsdk/gcloud_main.py", line 151, in CreateCLI
    generated_cli = loader.Generate()
  File "/Users/bahar/Downloads/google-cloud-sdk/lib/googlecloudsdk/calliope/cli.py", line 504, in Generate
    cli = self.__MakeCLI(top_group)
  File "/Users/bahar/Downloads/google-cloud-sdk/lib/googlecloudsdk/calliope/cli.py", line 674, in __MakeCLI
    log.AddFileLogging(self.__logs_dir)
  File "/Users/bahar/Downloads/google-cloud-sdk/lib/googlecloudsdk/core/log.py", line 1039, in AddFileLogging
    _log_manager.AddLogsDir(logs_dir=logs_dir)
  File "/Users/bahar/Downlo

In [18]:
from google.cloud import bigquery

In [19]:
# configure project info and bigquery client
project_id='syntheticlethality'

# construct a BigQuery client object.
client = bigquery.Client(project_id)

In [7]:
dataset_name='DepMap_public_20Q3'
dataset_description="""  
This DepMap release contains data from CRISPR knockout 
screens from project Achilles, as well as genomic characterization data from the CCLE project.
"""

CreateDataSet(client, dataset_name, project_id, dataset_description)

Dataset DepMap_public_20Q3 already exists.


In [8]:
#CCLE_mutation_data.csv file
mutation_data=pd.read_csv("https://ndownloader.figshare.com/files/24613355", sep="\t") 
#https://ndownloader.figshare.com/files/24613355

#sample_info.csv file
sample_info=pd.read_csv("https://ndownloader.figshare.com/files/24613394",  sep=",")
#https://ndownloader.figshare.com/files/24613394

#WES_SNP_CN_data.csv file
cn_data=pd.read_csv("https://ndownloader.figshare.com/files/24613352", index_col=0)
#https://ndownloader.figshare.com/files/24613352

#CCLE_expression.csv
gene_exp_data=pd.read_csv("https://ndownloader.figshare.com/files/24613325", index_col=0)
#https://ndownloader.figshare.com/files/24613325

#D2_Achilles_gene_effect.csv file
achilles_gene_effect=pd.read_csv("https://ndownloader.figshare.com/files/24613292", index_col=0)
#https://ndownloader.figshare.com/files/24613292


/Users/bahar/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (19,29,30) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [20]:
pwd

'/Users/bahar/Downloads/SL-Cloud-main2/DEPMAP_save_data'

In [21]:
# get annotations from the excel file 
depmap_annotations=pd.ExcelFile("../data/DEPMAP_20Q3_annotation.xlsx")

In [22]:
depmap_annotations.sheet_names 

['Export Summary',
 'Ref',
 'Sample_Info',
 'CCLE_Mutations',
 'CCLE_Gene_Expression',
 'CCLE_Copy_Number',
 'Achilles_Gene_Effect']

In [23]:
mutation_annotation=depmap_annotations.parse('CCLE_Mutations')
sample_annotation=depmap_annotations.parse('Sample_Info')
achilles_gene_effect_annotation=depmap_annotations.parse('Achilles_Gene_Effect')
CCLE_expression_annotation=depmap_annotations.parse('CCLE_Gene_Expression')
cnv_annotations=depmap_annotations.parse('CCLE_Copy_Number')

In [24]:
mutation_table_name='CCLE_mutation'
mutation_table_desc='''Pipeline: Mutations
MAF of gene mutations.'''

mutation_dict=mutation_annotation.to_dict('records')
mutation_table=CreateTable(client, mutation_data, dataset_name, mutation_table_name, project_id,  mutation_table_desc, mutation_dict)

1it [02:42, 162.30s/it]


Table created successfully


In [25]:
sample_info_table_name="sample_info"
sample_info_table_desc='''Cell line information definitions.'''

sample_dict=sample_annotation.to_dict('records')
CreateTable(client, sample_info, dataset_name, sample_info_table_name, project_id, sample_info_table_desc, sample_dict)

1it [00:03,  3.20s/it]


Table created successfully


In [30]:
cn_table_desc='''Pipeline: Copy number
Gene level copy number data, log2 transformed with a pseudo count of 1. 
This is generated by mapping genes onto the segment level calls.
.'''

cnv_long_format=CRISPRPreprocess(cn_data, 'CNA')
cnv_table_name="CCLE_gene_cn"
cnv_dict=cnv_annotations.to_dict('records')
CreateTable(client, cnv_long_format, dataset_name, cnv_table_name, project_id, cn_table_desc, cnv_dict)

1it [17:09, 1029.67s/it]


Table created successfully


In [31]:
CCLE_expression_table_desc='''
Pipeline: Expression
Ransom TPM gene expression data for just protein coding genes using RSEM. 
Log2 transformed, using a pseudo-count of 1.
'''
CCLE_expression_long_format=CRISPRPreprocess(gene_exp_data, 'TPM')
CCLE_expression_table_name="CCLE_gene_expression"

CCLE_expression_dict=CCLE_expression_annotation.to_dict('records')
CreateTable(client, CCLE_expression_long_format, dataset_name, CCLE_expression_table_name, project_id, CCLE_expression_table_desc, CCLE_expression_dict)




1it [07:51, 471.67s/it]


Table created successfully


In [32]:
achilles_gene_effect_table_desc='''Pipeline: Achilles_Post-CERES_
CERES data with principle components strongly related to known batch effects removed,
then shifted and scaled per cell line so the median nonessential KO effect is 0 and 
the median essential KO effect is -1.
'''
achilles_gene_effect_long_format=CRISPRPreprocess(achilles_gene_effect, 'Gene_Effect')
achilles_gene_effect_table_name="Achilles_gene_effect"
achilles_gene_effect_dict=achilles_gene_effect_annotation.to_dict('records')
CreateTable(client, achilles_gene_effect_long_format, dataset_name, achilles_gene_effect_table_name, project_id, achilles_gene_effect_table_desc, achilles_gene_effect_dict)



1it [05:42, 342.90s/it]


Table created successfully
